# 🚞 Zero-shot RE Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# # if you're running this in a colab notebook, you can run this cell to install the necessary dependencies
# !git clone https://github.com/jackboyla/GLiREL.git
# !cd GLiREL && pip install -e .
# !python -m spacy download en_core_web_sm

In [3]:
from glirel import GLiREL

save_path = 'logs/few_rel/few_rel-2024-05-31__08-29-45/model_9000'
model = GLiREL.from_pretrained(save_path, map_location='cpu')
# model

config.json not found in /home/jackboylan/GLiREL/logs/few_rel/few_rel-2024-05-31__08-29-45/model_9000
2024-06-01 01:04:23,368 - huggingface_hub.hub_mixin - WARNING - config.json not found in /home/jackboylan/GLiREL/logs/few_rel/few_rel-2024-05-31__08-29-45/model_9000
/opt/conda/envs/glirel/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/opt/conda/envs/glirel/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you 

# Inference

To infer, the model needs `tokens`, `NER`, and `zero shot labels`.

### Eval data

In [5]:
import json
with open('./data/few_rel_all.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

i = 0

tokens = data[i]['tokenized_text']
ner = data[i]['ner']
labels = list(set([r['relation_text'] for r in data[i]['relations']]))
print(tokens)
print()
print(ner)
print(labels)

['Derren', 'Nesbitt', 'had', 'a', 'history', 'of', 'being', 'cast', 'in', '"', 'Doctor', 'Who', '"', ',', 'having', 'played', 'villainous', 'warlord', 'Tegana', 'in', 'the', '1964', 'First', 'Doctor', 'serial', '"', 'Marco', 'Polo', '"', '.']

[[26, 28, 'Q2989881', 'Marco Polo'], [22, 24, 'Q2989412', 'First Doctor']]
['characters']


In [6]:
labels = ['country of origin', 'licensed to broadcast to', 'father', 'followed by'] + labels
print(labels)

['country of origin', 'licensed to broadcast to', 'father', 'followed by', 'characters']


In [7]:
relations = model.predict_relations(tokens, labels, threshold=0.01, ner=ner)

print('Number of relations:', len(relations))  # num entity pairs (both directions) * num classes.... provided they're over the threshold

sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
print("\nDescending Order by Score:")
for item in sorted_data_desc:
    print(item)

Number of relations: 1

Descending Order by Score:
{'head_pos': [26, 28], 'tail_pos': [22, 24], 'head_text': ['Marco', 'Polo'], 'tail_text': ['First', 'Doctor'], 'label': 'characters', 'score': 0.9807606935501099}


### Real-world example

In [8]:
# Real-world example
import spacy
nlp = spacy.load('en_core_web_sm')

text = "Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976. The company is headquartered in Cupertino, California."

# text = "Jack Dorsey's father, Tim Dorsey, is a licensed pilot. Jack met his wife Sarah Paulson in New York in 2003. They have one son, Edward."

labels = [
    'co-founder', 
    'country of origin', 
    'licensed to broadcast to', 
    'no relation', 
    'parent', 
    'followed by', 
    'located in or next to body of water', 
    'spouse', 
    'child', 
    'founder', 
    'headquartered in', 
    'acquired by', 
    'subsidiary of'
    ]



def predict_and_show(text, labels):
    doc = nlp(text)

    tokens = [token.text for token in doc]

    ner = [[ent.start, ent.end, ent.label_, ent.text] for ent in doc.ents]
    print(f"Entities detected: {ner}")

    relations = model.predict_relations(tokens, labels, threshold=0.01, ner=ner)

    print('Number of relations:', len(relations))

    sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
    print("\nDescending Order by Score:")
    for item in sorted_data_desc:
        print(f"{item['head_text']} --> {item['label']} --> {item['tail_text']} | score: {item['score']}")

predict_and_show(text, labels)

Entities detected: [[0, 2, 'ORG', 'Apple Inc.'], [5, 7, 'PERSON', 'Steve Jobs'], [8, 10, 'PERSON', 'Steve Wozniak'], [12, 14, 'PERSON', 'Ronald Wayne'], [15, 17, 'DATE', 'April 1976'], [23, 24, 'GPE', 'Cupertino'], [25, 26, 'GPE', 'California']]
Number of relations: 80

Descending Order by Score:
['Steve', 'Wozniak'] --> headquartered in --> ['California'] | score: 0.9892545938491821
['Steve', 'Jobs'] --> headquartered in --> ['California'] | score: 0.9890961050987244
['Ronald', 'Wayne'] --> headquartered in --> ['California'] | score: 0.9890897870063782
['Apple', 'Inc.'] --> headquartered in --> ['California'] | score: 0.9884379506111145
['April', '1976'] --> headquartered in --> ['California'] | score: 0.9883624911308289
['Steve', 'Wozniak'] --> headquartered in --> ['Cupertino'] | score: 0.9531661868095398
['Steve', 'Jobs'] --> headquartered in --> ['Cupertino'] | score: 0.9524778723716736
['Ronald', 'Wayne'] --> headquartered in --> ['Cupertino'] | score: 0.9524572491645813
['April

In [9]:
text = "Jack knows Gill. They live in the same house in London. They are not related."
labels = ['family relation', 'knows', 'lives with', 'loves', 'licensed to broadcast to', 'father', 'followed by', 'no relation', 'lives in',]
predict_and_show(text, labels)

Entities detected: [[0, 1, 'PERSON', 'Jack'], [2, 3, 'PERSON', 'Gill'], [11, 12, 'GPE', 'London']]
Number of relations: 8

Descending Order by Score:
['Gill'] --> lives in --> ['London'] | score: 0.9845479130744934
['Jack'] --> lives in --> ['London'] | score: 0.9842474460601807
['Jack'] --> lives with --> ['Gill'] | score: 0.33038315176963806
['Gill'] --> lives with --> ['Jack'] | score: 0.23151880502700806
['Jack'] --> knows --> ['Gill'] | score: 0.09202353656291962
['Gill'] --> knows --> ['Jack'] | score: 0.06196396425366402
['Jack'] --> no relation --> ['Gill'] | score: 0.028537264093756676
['Gill'] --> no relation --> ['Jack'] | score: 0.02333051897585392


In [10]:
import spacy
from spacy.tokens import Span
from spacy import displacy

def visualize_relation(text, relations):
    nlp = spacy.blank("en")
    doc = nlp(text)

    # Manually set dependency relations to visualize relations
    for token in doc:
        token.dep_ = "dep"  # default to 'dep'

    spans = []
    # Calculate character offsets for each entity
    for rel in relations:
        head = Span(doc, rel['head_pos'][0], rel['head_pos'][1], label=rel['head_text'])
        tail = Span(doc, rel['tail_pos'][0], rel['tail_pos'][1], label=rel['tail_text'])

        doc.ents += (head, tail)

        # Mock dependencies
        head_root = head.root
        tail_root = tail.root

        head_root.dep_ = "rel"  # Relation type can be customized
        head_root.head = tail_root  # Point head to tail

    options = {"fine_grained": True}
    displacy.render(doc, style="dep", options=options, jupyter=True)

# Example data
text = "Binsey located in or next to body of water River Thames"
relations = [
    {'head_pos': [0, 1], 'tail_pos': [9, 11], 'head_text': 'Binsey', 'tail_text': 'River Thames', 'label': 'located in or next to body of water', 'score': 0.9235768914222717},
    # {'head_pos': [9, 11], 'tail_pos': [0, 1], 'head_text': 'River Thames', 'tail_text': 'Binsey', 'label': 'located in or next to body of water', 'score': 0.12615662813186646}
]

visualize_relation(text, relations)
